In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from math import sqrt
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import contextily as ctx
import json
import base64
from io import BytesIO
from pyproj import Transformer
from shapely.geometry import LineString, box
from dash import Dash, html

In [2]:
# Variables globales

# Número de filas a leer del archivo CSV
nrows = 100000
# Ruta del archivo CSV
filename = "../train_data/taxis_trajectory/train.csv"

Esta segunda version de el filtro tasda unos 30-40 segundos entre habrir el archivo y filtrar 100000 datos.

In [3]:
def filtrar_coordenadas_anomalas(polyline, umbral_distancia):
    # Filtrar puntos que discorden significativamente de sus predecesores en la polilínea.
    if len(polyline) < 2:
        # Si hay menos de dos puntos, no hay suficiente información para filtrar
        return polyline

    puntos_filtrados = [polyline[0]]  # Mantener el primer punto siempre

    for i in range(1, len(polyline)):
        x1, y1 = puntos_filtrados[-1]
        x2, y2 = polyline[i]
        distancia = sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
        if distancia <= umbral_distancia:
            puntos_filtrados.append(polyline[i])

    return puntos_filtrados

In [4]:
def load_and_simplify_data(filename, rows, tolerance=0.001, umbral_distancia=0.01):
    # Cargar datos
    df = pd.read_csv(filename, nrows=rows, sep=",", low_memory=False)
    
    # Filtrar y crear LineString para cada polilínea
    def create_line(x):
        points = json.loads(x)
        points_filtrados = filtrar_coordenadas_anomalas(points, umbral_distancia)
        if len(points_filtrados) > 1:
            return LineString(points_filtrados)
        return None
    
    df['geometry'] = df['POLYLINE'].apply(create_line)
    
    # Eliminar filas con geometrías nulas
    df = df[df['geometry'].notnull()]
    
    # Convertir a Geopandas DataFrame
    gdf = gpd.GeoDataFrame(df, geometry='geometry')
    
    # Simplificar las geometrías
    gdf['geometry'] = gdf['geometry'].simplify(tolerance)
    
    return gdf

In [5]:
def filter_data_in_area(gdf, minx, miny, maxx, maxy):
    # Crear un polígono de área de interés
    area_of_interest = box(minx, miny, maxx, maxy)
    
    # Filtrar los datos para incluir solo aquellos completamente dentro del área de interés
    gdf_filtered = gdf[gdf.geometry.within(area_of_interest)]
    
    return gdf_filtered

In [6]:
def lisr_coordinates(gdf):   
    # Crear listas vacías para las coordenadas x e y
    x_coords_flat = []
    y_coords_flat = []

    # Iterar sobre cada geometría en el GeoDataFrame
    for geom in gdf['geometry']:
        # Verificar que la geometría sea una LineString
        if isinstance(geom, LineString):
            # Iterar sobre cada punto en la LineString
            for point in geom.coords:
                x_coords_flat.append(point[0])  # Añadir la coordenada x a la lista
                y_coords_flat.append(point[1])  # Añadir la coordenada y a la lista
    
    return x_coords_flat, y_coords_flat

In [7]:
def solicitar_coordenadas(gdf):
    """ print("Por favor, introduce las coordenadas para el área de interés.")
    minx = float(input("Introduce la longitud mínima (minx): "))
    miny = float(input("Introduce la latitud mínima (miny): "))
    maxx = float(input("Introduce la longitud máxima (maxx): "))
    maxy = float(input("Introduce la latitud máxima (maxy): ")) """
    
    """ minx=-8.689
    miny=41.107
    maxx=-8.560
    maxy=41.185 """

    x_coords_flat, y_coords_flat = lisr_coordinates(gdf)

    maxx, maxy, minx, miny = max(x_coords_flat), max(y_coords_flat), min(x_coords_flat), min(y_coords_flat)

    return minx, miny, maxx, maxy 

In [8]:
# Esto aun no se como pedirlo o si hacerlo
def solicitar_map_position():
    bin_count = 300 # Cantidad de bins para el histograma 2D
    posicion_x = 0 # Coordenada x del centro del mapa, para despazarlo por el eje y, poner valore arededro de 1000
    posicion_y = 0 # Coordenada y del centro del mapa
    zoom = 2 # Nivel de zoom del mapa, por cuanto se divide las cordenadas

    return bin_count, posicion_x, posicion_y, zoom

In [9]:
def map_ilustration(gdf, minx, miny, maxx, maxy):
    gdf = gdf.set_crs("EPSG:4326")

    # Luego, usar estas coordenadas en la función de filtrado
    gdf = filter_data_in_area(gdf, minx, miny, maxx, maxy)

    # Transforcion en EPSG:3857 para alinear con el mapa base de Contextily
    gdf = gdf.to_crs(epsg=3857)
    
    # Crear una figura con Matplotlib
    fig, ax = plt.subplots(figsize=(10, 10), dpi=300)
    gdf.plot(ax=ax, linewidth=0.5, color='blue')

    # Añadir un mapa base con Contextily
    ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)

    # Añadir título y etiquetas
    plt.title('Mapa de Trayectorias de Taxis con Mapa de Fondo')
    plt.xlabel('Longitud')
    plt.ylabel('Latitud')

    # Crear un objeto BytesIO para guardar la imagen
    img_data = BytesIO()
    plt.savefig(img_data, format='png')
    img_data.seek(0)  # Mover el 'cursor' al principio del archivo en memoria
    
    # Es importante cerrar la figura para liberar memoria
    plt.close(fig)

    # Codificar la imagen generada en base64
    encoded_string = base64.b64encode(img_data.read()).decode('utf-8')

    return encoded_string

In [10]:
def map_heat(gdf, minx, miny, maxx, maxy, bin_count, posicion_x, posicion_y, zoom):  
    # Obtener las coordenadas x e y de las geometrías 
    x_coords_flat, y_coords_flat = lisr_coordinates(gdf)

    # Calcular el histograma bidimensional de las coordenadas x e y
    heatmap, _, _ = np.histogram2d(x_coords_flat, y_coords_flat, bins=bin_count, density=True, range=[[minx, maxx], [miny, maxy]])

    # Inicializar el transformador de coordenadas
    transformer = Transformer.from_crs("epsg:4326", "epsg:3857", always_xy=True)

    # Transformar las coordenadas
    xmin, ymin = transformer.transform(minx, miny)
    xmax, ymax = transformer.transform(maxx, maxy)
    
    """ # Calcular el centro y el rango de los ejes x e y
    x_center, y_center = ((xmin + xmax) / 2) + posicion_x, ((ymin + ymax) / 2) + posicion_y
    x_range, y_range = (xmax - xmin) / zoom, (ymax - ymin) / zoom """

    # Crear la figura y los ejes para matplotlib
    fig, ax = plt.subplots(figsize=(10, 10), dpi=300)

    # Crear una normalización logarítmica
    norm = colors.LogNorm(vmin=heatmap.min()+1, vmax=heatmap.max())

    # Mostrar el mapa de calor y capturar el objeto mappable retornado por imshow
    mappable = ax.imshow(heatmap.T, origin='lower', norm=norm ,extent=[xmin, xmax, ymin, ymax], aspect='auto', alpha=0.7, zorder=2)

    # Añadir el mapa base
    ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron, zoom='auto')

    """ # Ajustar los límites de los ejes para coincidir con los bordes del histograma
    ax.set_xlim(x_center - x_range, x_center + x_range)
    ax.set_ylim(y_center - y_range, y_center + y_range) """

    # Añadir barra de color, títulos y etiquetas usando el objeto mappable
    plt.colorbar(mappable, label='Densidad')
    plt.title('Mapa de Calor de Trayectorias de Taxis con Mapa de Fondo')
    plt.xlabel('Longitud')
    plt.ylabel('Latitud')

    # Crear un objeto BytesIO para guardar la imagen
    img_data = BytesIO()
    plt.savefig(img_data, format='png')
    img_data.seek(0)  # Mover el 'cursor' al principio del archivo en memoria
    
    # Es importante cerrar la figura para liberar memoria
    plt.close(fig)

    # Codificar la imagen generada en base64
    encoded_string = base64.b64encode(img_data.read()).decode('utf-8')

    return encoded_string

El tiempo aumente cuando se reducen la cordenadas a un lugar mas concreto, el archivo completo tarda unos 7 min y mientras que centrado en oporto tarde 14 min.

In [11]:
# Cargar y simplificar datos
gdf = load_and_simplify_data(filename, nrows)

In [12]:
# solicitar coordenadas
minx, miny, maxx, maxy = solicitar_coordenadas(gdf)

# solicitar datos del mapa
bin_count, posicion_x, posicion_y, zoom = solicitar_map_position()

In [13]:
# Crear un mapa ilustrativo
html_map = map_ilustration(gdf, minx, miny, maxx, maxy)

# Crear un mapa de calor
html_heatmap = map_heat(gdf, minx, miny, maxx, maxy, bin_count, posicion_x, posicion_y, zoom)

In [14]:
# Crear aplicación Dash
app = Dash(__name__)

In [15]:
# Configurar el layout de la aplicación Dash
app.layout = html.Div([
    # División para el título
    html.Div([
        html.H1("Visualización de Datos de Taxi")
    ], style={'textAlign': 'center'}),  # Centrar el título

    # Contenedor para los mapas, uno encima del otro
    html.Div([
        # División para la imagen del gráfico
        html.Div([
            html.Img(src='data:image/png;base64,{}'.format(html_map), style={'width': '100%'})
        ], style={'width': '50%', 'textAlign': 'left', 'display': 'inline-block'}),  # Ajustar el ancho y alinear a la izquierda

        # División para la imagen heatmap
        html.Div([
            html.Img(src='data:image/png;base64,{}'.format(html_heatmap), style={'width': '100%'})
        ], style={'width': '50%', 'textAlign': 'left', 'display': 'inline-block'})  # Ajustar el ancho y alinear a la izquierda
    ], style={'width': '100%'})
])


In [ ]:
# Ejecutar la aplicación
if __name__ == '__main__':
    app.run_server(debug=True, host='127.0.0.1', port=8050)
    # http://127.0.0.1:8050/